In [1]:
%load_ext autoreload
%autoreload
%matplotlib inline

from news_crawler import NewsCrawler, select_urls
from IPython.display import display
from IPython.core.debugger import set_trace

In [2]:
crawler = NewsCrawler()

# 1. URL을 모은다

In [3]:
# 제거고려: nextbigfuture, nbcnews, joongang, chosun

In [6]:
collect_result = crawler.collect(); display(*collect_result)

100.00% (328.55 seconds): Collecting URLs... indiatimes          

100.00% (37.44 seconds): Selecting URLs... cryptonews          

,collected,collected_unique,selected
all,25311,25247,1507
euronews,1085,1085,210
reuters,1150,1150,117
yahoo,369,369,114
dailymail,3119,3119,95
businessinsider,634,634,89
rt,401,401,79
metro,244,244,60
breitbart,385,385,60
aljazeera,472,472,49


,url,pubs,actual_pub
0,https://bbc.com/news/uk-politics-50115151,"bbc, realclearpolitics",bbc
1,https://bbc.com/news/world-asia-china-50115629,"bbc, axios",bbc
2,https://businessinsider.co.za/dramatic-satelli...,"businessinsider, news24",businessinsider
3,https://foxbusiness.com/media/verizon-looking-...,"fox, realclearpolitics",fox
4,https://foxbusiness.com/money/andy-puzder-here...,"fox, realclearpolitics",fox
5,https://foxnews.com/world/pelosi-other-us-lawm...,"fox, realclearpolitics",fox
6,https://hotair.com/archives/john-s-2/2019/10/1...,"townhall, hotair",hotair
7,https://huffpost.com/entry/alexandria-ocasio-c...,"huffpost, realclearpolitics",huffpost
8,https://msnbc.com/mtp-daily/watch/engel-from-s...,"msnbc, nbcnews",msnbc
9,https://msnbc.com/mtp-daily/watch/full-mcgurk-...,"msnbc, nbcnews",msnbc


# 2. 해당 URL의 뉴스를 다운로드한다

In [7]:
download_result = crawler.download(); download_result

100.00% (1681.95 seconds): Downloading... euronews            

,downloaded,trashed,total
all,870,637,1507
euronews,19,191,210
reuters,51,66,117
yahoo,92,22,114
dailymail,72,23,95
businessinsider,73,16,89
rt,11,68,79
metro,57,3,60
breitbart,58,2,60
aljazeera,1,48,49
